In [ ]:
from abc import ABCMeta 
from abc import abstractmethod
from abc import abstractproperty

from tqdm import tqdm_notebook

import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

import pandas

from matplotlib import pyplot as plt
%matplotlib inline

## Bernoulli Bandit

We are going to implement several exploration strategies for simplest problem - bernoulli bandit.

The bandit has $K$ actions. Action produce 1.0 reward $r$ with probability $0 \le \theta_k \le 1$ which is unknown to agent, but fixed over time. Agent's objective is to minimize regret over fixed number $T$ of action selections:

$$\rho = T\theta^* - \sum_{t=1}^T r_t$$

Where $\theta^* = \max_k\{\theta_k\}$

**Real-world analogy:**

Clinical trials - we have $K$ pills and $T$ ill patient. After taking pill, patient is cured with probability $\theta_k$. Task is to find most efficient pill.

A research on clinical trials - https://arxiv.org/pdf/1507.08025.pdf

In [ ]:
class BernoulliBandit:
    """
    Implementation of the Bernoulli Bandit
    """
    
    def __init__(self, n_actions=5):
        """
        Constructor which sets the probability for reward
        
        Parameters
        ----------
        n_actions : int
            How many action we are allowed to take
        """
        self._probs = np.random.random(n_actions)
    
    # NOTE: The property decorator makes this function accessible as an attribute
    #       https://www.python-course.eu/python3_properties.php
    @property
    def action_count(self):
        """
        Counts the number of actions
        
        Returns
        -------
        int
            The number of actions
        """
        return len(self._probs)
    
    def pull(self, action):
        """
        Function for "pulling the lever"
        
        If the action probability is larger tha a random drawn number,
        a reward of 1 is returned
        
        Parameters
        ----------
        action : int
            The action to take (aka the lever to pull)
        
        Returns
        -------
        float
            The reward (either 0.0 or 1.0)
        """
        if np.random.random() > self._probs[action]:
            return 0.0
        return 1.0
    
    def optimal_reward(self):
        """
        Returns the highest probability of all the actions
        
        This is used for regret calculation
        
        Returns
        -------
        float
            The maximum probability of all the actions
        """
        return np.max(self._probs)
    
    def step(self):
        """ 
        Used in nonstationary version
        """
        pass
    
    def reset(self):
        """ 
        Used in nonstationary version
        """
        pass

In [ ]:
class AbstractAgent(metaclass=ABCMeta):
    """
    The abstraction of the agents
    """
    
    def init_actions(self, n_actions):
        """
        Initialization of the actions
        
        Parameters
        ----------
        n_actions : int
            The number of possible actions
        """
        
        self._successes = np.zeros(n_actions)
        self._failures = np.zeros(n_actions)
        self._total_pulls = 0
    
    # NOTE: The abstractmethod decorator tells the child classes that this method must be defined
    #       https://www.python-course.eu/python3_abstract_classes.php
    @abstractmethod
    def get_action(self):
        """
        Get current best action
        
        The implementations should return an integer corresponding to the best action
        """
        pass
    
    def update(self, action, reward):
        """
        Observe reward from action and update agent's internal parameters
        
        Parameters
        ----------
        action : int
            The action taken
        reward : int
            The obtained return from the action
        """
        self._total_pulls += 1
        if reward == 1:
            self._successes[action] += 1
        else:
            self._failures[action] += 1
    
    @property
    def name(self):
        """
        Returns the class name
        
        Returns
        -------
        str
            The class name
        """
        return self.__class__.__name__

In [ ]:
class RandomAgent(AbstractAgent):
    """
    The implementation of a random agent, which always chooses a random action
    """
    
    def get_action(self):
        """
        Returns a random action
        
        Return
        ------
        int
            The chosen action
        """
        return np.random.randint(0, len(self._successes))

### Epsilon-greedy agent

> **for** $t = 1,2,...$ **do**
>> **for** $k = 1,...,K$ **do**
>>> $\hat\theta_k \leftarrow \alpha_k / (\alpha_k + \beta_k)$

>> **end for** 

>> $x_t \leftarrow argmax_{k}\hat\theta$ with probability $1 - \epsilon$ or random action with probability $\epsilon$

>> Apply $x_t$ and observe $r_t$

>> $(\alpha_{x_t}, \beta_{x_t}) \leftarrow (\alpha_{x_t}, \beta_{x_t}) + (r_t, 1-r_t)$

> **end for**

Implement the algorithm above in the cell below:

In [ ]:
class EpsilonGreedyAgent(AbstractAgent):
    """
    The implementation of the epsilon greedy agent
    """
    
    def __init__(self, epsilon=0.01):
        """
        Sets the epsilon
        
        Parameters
        ----------
        epsilon : float
            The chance of taking a random action
        """
        
        self._epsilon = epsilon

    def get_action(self):
        """
        Get the action to take in an epsilon-greedy fashion
        
        Returns
        -------
        action : int
            The chosen action
        """
        
        epsilon = self._epsilon
        n_actions = len(self._successes)
        
        # NOTE: Recall that self._success and self._failures are
        #       arrays that counts how many times an action resulted in
        #       a success and when it resulted in a failure
        alpha = self._successes
        beta = self._failures
        
        # NOTE: Add machine epsilon to avoid division by 0
        theta_hat = alpha/(alpha + beta + np.finfo(float).eps)
        
        x = np.argmax(theta_hat)
        
        # Choose action according to the epsilon-greedy strategy
        possibilities = [x] + list(range(n_actions))
        probabilities = [1-epsilon] + [epsilon/n_actions]*n_actions
        action = np.random.choice(possibilities, p=probabilities)
        
        return action
        
    @property
    def name(self):
        """
        Returns the class name with the epsilon appended
        
        Returns
        -------
        str
            The class name with the value of epsilon appended
        """
        return self.__class__.__name__ + "(epsilon={})".format(self._epsilon) 

### UCB Agent
Epsilon-greedy strategy heve no preference for actions. It would be better to select among actions that are uncertain or have potential to be optimal. One can come up with idea of index for each action that represents otimality and uncertainty at the same time. One efficient way to do it is to use UCB1 algorithm:

> **for** $t = 1,2,...$ **do**
>> **for** $k = 1,...,K$ **do**
>>> $w_k \leftarrow \alpha_k / (\alpha_k + \beta_k) + \sqrt{2log\ t \ / \ (\alpha_k + \beta_k)}$

>> **end for** 

>> $x_t \leftarrow argmax_{k}w$

>> Apply $x_t$ and observe $r_t$

>> $(\alpha_{x_t}, \beta_{x_t}) \leftarrow (\alpha_{x_t}, \beta_{x_t}) + (r_t, 1-r_t)$

> **end for**


__Note:__ in practice, one can multiply $\sqrt{2log\ t \ / \ (\alpha_k + \beta_k)}$ by some tunable parameter to regulate agent's optimism and wilingness to abandon non-promising actions.

More versions and optimality analysis - https://homes.di.unimi.it/~cesabian/Pubblicazioni/ml-02.pdf

In [ ]:
class UCBAgent(AbstractAgent):
    """
    The implementation of the UCB-1 agent
    """
    
    def get_action(self):
        """
        Get the action to take in an UCB-1 fashion
        
        Returns
        -------
        action : int
            The chosen action
        """
                
        # NOTE: Recall that self._success and self._failures are
        #       arrays that counts how many times an action resulted in
        #       a success and when it resulted in a failure
        alpha = self._successes
        beta = self._failures
        
        # NOTE: t is a scalar
        t = self._total_pulls
        
        # NOTE: Adding machine epsilons to avoid division by zero
        # NOTE: Adding 1 in the log in order to not take sqrt of a negative number
        w = alpha/(alpha + beta + np.finfo(float).eps) +\
            np.sqrt(2*np.log(t+1)/(alpha + beta + np.finfo(float).eps))
        
        x = np.argmax(w)
        
        return x
    
    @property
    def name(self):
        """
        Returns the class name
        
        Returns
        -------
        str
            The class name
        """
            
        return self.__class__.__name__

### Thompson sampling

UCB1 algorithm does not take into account actual distribution of rewards. If we know the distribution - we can do much better by using Thompson sampling:

> **for** $t = 1,2,...$ **do**
>> **for** $k = 1,...,K$ **do**
>>> Sample $\hat\theta_k \sim beta(\alpha_k, \beta_k)$

>> **end for** 

>> $x_t \leftarrow argmax_{k}\hat\theta$

>> Apply $x_t$ and observe $r_t$

>> $(\alpha_{x_t}, \beta_{x_t}) \leftarrow (\alpha_{x_t}, \beta_{x_t}) + (r_t, 1-r_t)$

> **end for**
 

More on Tompson Sampling:
https://web.stanford.edu/~bvr/pubs/TS_Tutorial.pdf

In [ ]:
class ThompsonSamplingAgent(AbstractAgent):
    """
    The implementation of the Thompson sampling agent
    """
    
    def get_action(self):
        """
        Get the action to take in a Thompson sampling fashion
        
        Returns
        -------
        action : int
            The chosen action
        """
        
                        
        # NOTE: Recall that self._success and self._failures are
        #       arrays that counts how many times an action resulted in
        #       a success and when it resulted in a failure
        alpha = self._successes
        beta = self._failures
        
        # NOTE: We add 1 to all elements as alpha and beta < 0 in the beta distribution
        p = np.random.beta(alpha+1, beta+1)
        
        return np.argmax(p)
    
    @property
    def name(self):
        """
        Returns the class name
        
        Returns
        -------
        str
            The class name
        """
        
        return self.__class__.__name__

In [ ]:
from collections import OrderedDict

def get_regret(env, agents, n_steps=5000, n_trials=50):
    """
    Retruns the regret after n_trails
    
    Parameters
    ----------
    env : BernoulliBandit-like
        The environment to play with
    agent : AbstractAgent-like
        An agent following the AbstractAgent
    n_steps : int
        How many times to pull the levels per trial
    n_trials : int
        How many trials to run
    
    Returns
    -------
    scores : OrderedDict
        An ordered dict with the agent name as a key and the 
        normalized cumulative sum of the regret
    """
    
    scores = OrderedDict({
        agent.name : [0.0 for step in range(n_steps)] for agent in agents
    })

    for trial in tqdm_notebook(range(n_trials), desc='Trial'):
        env.reset()
        
        for a in agents:
            a.init_actions(env.action_count)

        for i in tqdm_notebook(range(n_steps), desc='Step', leave=False):
            optimal_reward = env.optimal_reward()
            
            for agent in agents:
                action = agent.get_action()
                reward = env.pull(action)
                agent.update(action, reward)
                scores[agent.name][i] += optimal_reward - reward
                
            env.step()  # change bandit's state if it is unstationary

    for agent in agents:
        scores[agent.name] = np.cumsum(scores[agent.name]) / n_trials
    
    return scores

In [ ]:
def plot_regret(scores):
    """
    Plots the regret
    
    Parameters
    ----------
    scores : OrderedDict
        An ordered dict with the agent name as a key and the 
        normalized cumulative sum of the regret    
    """
    
    for agent in agents:
        plt.plot(scores[agent.name])

    plt.legend([agent for agent in scores])
    
    plt.ylabel("regret")
    plt.xlabel("steps")
    
    plt.show()

In [ ]:
# Uncomment agents
agents = [
     EpsilonGreedyAgent(),
     UCBAgent(),
     ThompsonSamplingAgent()
]

regret = get_regret(BernoulliBandit(), agents, n_steps=10000, n_trials=10)

In [ ]:
plot_regret(regret)

### Submit to coursera

In [ ]:
EMAIL = ''
TOKEN = ''

In [ ]:
from submit import submit_bandits

# NOTE: Due to randomness, it could be that you need to run get_regret a couple of times to get the scores to pass
submit_bandits(regret, EMAIL, TOKEN)